# Downloading URL Images from Appen Job Report
1. After obtaining full annotation job result report from Appen, store the csv file(ex: 'f1810360-2.csv') in your working directory(folder name: 'Appen-job-report').
2. We will create a dataframe to preprocess URLs and download them into images.
3. URLs from the report are only accessible for about 2-3 weeks, so you need to download a new report once URLs expire.

In [2]:
import pandas as pd
import urllib.request
import os
import shutil

In [7]:
pd.set_option('display.max_colwidth', -1) # For viewing full dataframe

# Retrieve API_key from your Appen account
API_key = 'Wsh6LNUfzVLvyceSMtSq'

# Write your file path for the Appen job report
FILENAME = '/data/Unet_data/Appen-job-report/v0.4-separating-glands-2-f1847066.csv'
df = pd.read_csv(FILENAME)

# URL preprocessing steps on "annotation" column to turn it into downloadable format:
df[["6","7","url","8"]] = df["annotation"].str.split(",", expand=True)
df[["9","10","11","annotation","12"]] = df["url"].str.split('"', expand=True)
df[["1","2","3","4","5","img"]] = df["image_url"].str.split("/", expand=True)
df['file_name'] = df['img'] + '_' + df['_worker_id'].astype(str)
df1 = df[['annotation','file_name']]

# Image name preprocessing to make image names into "image id + worker id" format.
start, stop, step = 0, -16, 1
s_file_name = df1['file_name'].str.slice(start, stop, step)

start, stop, step = -8, 36, 1
s_workerid = df1['file_name'].str.slice(start, stop, step)
s_new = s_file_name + s_workerid
df1['file_name'] = s_new

start, stop, step = 24, 500, 1 # set approximate maximum number(500) for len(column str)
s_anno = df1['annotation'].str.slice(start, stop, step)

# Below('annotation') is the URL format you need, in order access URLs and download them in your folder using for loop.
s_anno2 = 'https://api-beta.' + s_anno + "&key=" + API_key
df1['annotation'] = s_anno2
df1

<ipython-input-7-1252c9beb4cb>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1) # For viewing full dataframe
<ipython-input-7-1252c9beb4cb>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['file_name'] = s_new
<ipython-input-7-1252c9beb4cb>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['annotation'] = s_anno2


,annotation,file_name
0,https://api-beta.appen.com/v1/redeem_token?token=dRurkccGHaozqnCpXcGb0/yx6R2cEWIR+kqjO7TFsy4PakPiZfumbRXcjcO/V34HxWhqbpVT5Ams6ymGIlgU2U4S7pFzhitXtUg6m1YmKHlJR2pxU86Hpo8jUVMDQpl7lDcfyC+i1xCrFekuyYkjgh8tJCxezIxHyy+qxRKG7SJSHcVoyJfFHU771yKkzkj/fJOf08lVa0hrgNSKcXMTSnSnYh5Y4Zre8F+iTYhUdPRyi2si6bWNhxFppK+hj/qmAKGx8wCJqpzNQERj6XoaeakjAfYPw631Ld9rmS05ROc+3YtRB8i6mZCG1wlmSeSCe4h65YJAeZDlS0UmIg==--tadcJonO3eIW2Ihy--zTo7v86z+DKPotLmKmkV+g==&version=474&key=Wsh6LNUfzVLvyceSMtSq,2020_02_05_10_25_51_46197185
1,https://api-beta.appen.com/v1/redeem_token?token=rigfzo+SiZS0XDKFuqATXIh8wiDylKj9n92+s8RRKd9xGp3oWXh4Sqy5HK2qFsN2R3CRpykw1TDcri50cCjC7SkfKHX4Z2d3yuKu9mvCzZ95Rm0Wdqp39xXMeFc1xf4TB/2Aq5NPMP7tiem0m341PHQ47dvpcy7STACpzvATtWxlWfgko31cJGF2xEr6CWO+9kku/0IewNkGQooKvSBOwV/zQLLUPproQyBxmSSDjaQwuhd/Z3yow5Vx4iuuDm07wgoMUMkTYDZyVvOIxWmXPi1/wL2sj/bsDS9hbSp+rW4EU25wBB7KMpnI6usNYJn5AvNABdf/YJQRBXhA0g==--AqjDForfUNBQhQHN--Iwru6hI9qw0l8YoZi9g2eQ==&version=474&key=Wsh6LNUfzVLvyceSMtSq,2020_02_05_10_25_51_46067444
2,https://api-beta.appen.com/v1/redeem_token?token=8peS2NoSlpVqvmV/Cc4G1LNeBw2eD196D33UTGbdBfnoQ8r9g30xn5BsiMQpCAhZ9EKH91qzLRq98JSaJmF3ukeGphOPTO3mNQukNyeGECb3PKEmo9YBZ+cjHmFqox/5ZOXkP5nQNWbRO7hcY09kwyg2n3Dz8Qz0WIrknzJAF8jV5GJ48PH3ClQk/EaPjxD3ok1QAamoBRZcYqyBtYEBx+EOSV5J2+Xe9hGAqxy0/s8qnui76crPIMUan8tE1DWzkor0v3gPCgIXn1yuIyGIy4UVgEG+CgcM/8gpuvGbGlW78vquPzgNg7UaxAGSFW1AZRPvYqi6pEe1QvDS0A==--ziRJa+DzCxPlsVgi--1NrG4VtPbwb8UJ2Khte5/A==&version=474&key=Wsh6LNUfzVLvyceSMtSq,2020_02_05_10_25_51_46067465
3,https://api-beta.appen.com/v1/redeem_token?token=u7mAvsv7MMeUO48nGPNE0ExsnDsT/pUgUeGVjCN+XtLN4JPDvpcn4tfcJTWPzNoq8mxWmDt6m4TXFrekv42Qr0iOacclpkcBkcfXIXU7axVwzzbcSFcaVwQhAley5m5NfDGSp/P9wbKinj2+zZJZjAPhoY7KIb77ABgsZucjjyec70CNWe7mxJlAyICWgvYXRDgZ8u5RicmUpgO7XXBFBz0pEBdPDqYNaPIfZpTR8C3gZFiYxrHeBFm4dUMNLVdcGeHpokLFp8D4I89qB9bJo+RK+eDFB14k7rVWq2f7D9xfBSZ0us209wjbC0AHK7pQDyAKgqTZLhEZXRo4lg==--wDheFvs8/cWMmwMp--yjxtd6QK4CKblt1J8KVpUw==&version=474&key=Wsh6LNUfzVLvyceSMtSq,2020_02_05_10_25_51_46074771
4,https://api-beta.appen.com/v1/redeem_token?token=N82hpbikk/tgKk9w3gj1xzjAZW8hIpocz5CZ8ca+gTpQnafhbr7FP9Vl93sosRGS4E4HJA7L/MazoboU8czNDhucuMTFnDshyFsQRgQqxuZ6d5r4G8EHcsBco8MVnaulDAfX7htvBmY7n+xWYEJoyxp0CFjrzfGX4jCMFOmYpz1u/taUuMUEUSArtWrQCUfDLpyEHPXkYruIdNxV8rXIfiUAKjzflyraXStfprnjJYqqQC4wv13c/NRsPTw6YfE0EREzc/SAsrSkIZMv0VaKaGVQ7rWvDR/6gKcTUVSWYZhS3cixDtP2qSkVss1Y207XoIvEHuBCrA3L/PidtA==--1K2AZKrxqF1geppc--hOGTaixWaiGujPM1UCBiFg==&version=474&key=Wsh6LNUfzVLvyceSMtSq,2020_02_05_10_25_51_46547929
...,...,...
310,https://api-beta.appen.com/v1/redeem_token?token=yh4GEnc/RQJIXmlaCDt1+CpnLfpPlfnJSnWbsPPiV33lMW7O70jECIezmY+VUlfTuQlef/JQUNTt6PmIvcV+DfVHhVa572WesxiM0jYsqP/OjK7qKG9+iS8Y41bArxiQR9BX7Ht1BdtpbJRS6iYYK3jN2ryJED8tkrHQgWg8QypUfPt2yRwnjveqEW7EeAeF9UDvF//dMFLCaqJPrwPjG8pQMgebb2yILH/13OsltQeJ8rmtKxgqlYhhwqyfqBnNZz1dI90qQ+4wDrCawy3N8GdE69w2X3JlzNHzAq2J6wz5BVySRgCiRksZUsfPCJgGVHN+rGGWTiJWuVqO3A==--IlsfHZgvC/w31wDn--k9uFqahvITZ+WS4iDg9Bkg==&version=474&key=Wsh6LNUfzVLvyceSMtSq,2020_02_05_10_24_58_46374990
311,https://api-beta.appen.com/v1/redeem_token?token=Xbmo3vZVWf/CC1nlnwDEro7u1XO/+687y5eytSw0qiRuI2Q14WD4ahJLYuu+11/Wuc5UlScViN5e5EM9cpxyNsYUua+DG+FqIl0/TMp4iEsMqyOSqPV1lKXeI2Yjdk1WXaTUbg9cFYZnhmq8L8b3SS97qxgclnaunEIv5ck4gR0ZDF+nkal5rmavQFGQn7XPBuMALbLNfddPqponJPS4D9pqpgx/xIo2QncEpL5WI3CP6HJqDn58Wrd3v9STZ5QZXOxDJItFo+T9svqr/FmGMwygVPSwpHqyMbMXfJMwXbc9kiWFgKK4SaxRRolPX2gmfdvIj3QCl8bY9FCzHw==--gsa4v38YqaUvUjKQ--dyoQx3MIv4FEKGF2MolhXw==&version=474&key=Wsh6LNUfzVLvyceSMtSq,2020_02_05_10_24_58_46384083
312,https://api-beta.appen.com/v1/redeem_token?token=4rMj9ADB3SgG6hZqx13hRdLYYNnSCDSulwJlBHpIOOtqtIYf+c6knQlLPxCkra5F/gU9xfOkU4wPrCSOP3A0zfye12EqaZjFd9snJF+HqfyJFXewq1HVl7+Jxaqmd/Kia9gOA0BXTIuev+52tW0QXacQdINFKuMITW5NF3XOa7ATJwyvu4PpNrDX4ksMN0ql2NBZIeznPaWp4+mSlF3AfdiYtkgduHc4le/ZMZ/PKMjy3RlYXzXbMLMeGaBTqZPL3K6Cr95HqCf7ylGeiLl++PIhQbGBTqvOBRf7CkXdY8bpYWSFULubCptH3/UUczj8I4lxtwIjvC6m9PLBEQ==--gNqGRWKY7qLMV3d7--sKBJKrat+aFZnT0zRWyzLg==&version=47

In [10]:
#------------------------------------------------------
# A function to download each URL into an image

def url_to_jpg(i, url, file_path):
    
    filename = '{}.jpg'.format(i)
    full_path = '{}{}'.format(file_path, filename)
    urllib.request.urlretrieve(url, full_path)
    print('{} saved.'.format(filename))
    
    return None
#------------------------------------------------------

# Create a new folder to download these images
FILE_PATH = '/data/Unet_data/v0.4-separating-glands-2/y/'

for i, row in df1.iterrows():
    url_to_jpg(row[1], row[0], FILE_PATH)

2020_02_05_10_25_51_46197185.jpg saved.
2020_02_05_10_25_51_46067444.jpg saved.
2020_02_05_10_25_51_46067465.jpg saved.
2020_02_05_10_25_51_46074771.jpg saved.
2020_02_05_10_25_51_46547929.jpg saved.
2020_02_05_10_27_43_46067465.jpg saved.
2020_02_05_10_27_43_46067444.jpg saved.
2020_02_05_10_27_43_46197185.jpg saved.
2020_02_05_10_27_43_46374990.jpg saved.
2020_02_05_10_27_43_46197504.jpg saved.
2020_02_05_10_29_50_46197185.jpg saved.
2020_02_05_10_29_50_46067444.jpg saved.
2020_02_05_10_29_50_46067465.jpg saved.
2020_02_05_10_29_50_46074771.jpg saved.
2020_02_05_10_29_50_46547929.jpg saved.
2020_02_05_10_30_42_46067465.jpg saved.
2020_02_05_10_30_42_46067444.jpg saved.
2020_02_05_10_30_42_46197185.jpg saved.
2020_02_05_10_30_42_46374990.jpg saved.
2020_02_05_10_30_42_46546212.jpg saved.
2020_02_05_10_31_39_46067465.jpg saved.
2020_02_05_10_31_39_46067444.jpg saved.
2020_02_05_10_31_39_46197185.jpg saved.
2020_02_05_10_31_39_46374990.jpg saved.
2020_02_05_10_31_39_46434619.jpg saved.


In [11]:
# count=0
# for file in os.listdir(FILE_PATH):
#     print(file)
#     count+=1
# print("total: ", count)

In [13]:
# Remove non-image files(e.g. '.ipynb_checkpoints')
count=0
count_no_img=0
for file in os.listdir(FILE_PATH):
    count+=1
    if not file.endswith('jpg'):
        count_no_img+=1
        print(file)
        try:
            os.remove(FILE_PATH + '{}'.format(file))
        except:
            shutil.rmtree(FILE_PATH + '{}'.format(file))
        
print('total count:', count)
print('non-img count:', count_no_img)

total count: 315
non-img count: 0
